**Project #1 - dashboard #1**

Using the plotly and dash libraries, create a dashboard consisting of two visualizations:

- a line chart that shows the cumulative number of COVID-19 cases over time
- a line chart that shows the cumulative number of COVID-19 deaths over time

The dashboard should consist of:
- title that depends on the value of the filter (e.g. Cumulative number of positive cases in {country})
- title (label) of the filter
- filter by country, i.e. the user should be able to see charts only for the selected country
- two visualizations placed side-by-side (in one line)

To run the dash application in Google Colab, use the solution presented in the course, based on the threading and google.colab libraries. Use the function that displays the application in a new browser tab.

In [1]:
# Install seaborn, pandas, matplotlib, plotly, dash, and threading if necessary
!pip install pandas matplotlib plotly dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 26.0 MB/s eta 0:00:00


In [2]:
import dash
import pandas as pd
import numpy as np
import plotly.express as px
import threading
from dash import dcc, html, Input, Output
from google.colab import output, drive

drive.mount('/drive')

Mounted at /drive


1. Install necessary libraries.
2. Load your data into a DataFrame.
3. Define the layout of your Dash app.

DIV
- H1 - title based on the value of dropdown
- Label
- Dropdown (countries)

DIV
- DIV - H2, Graph
- DIV - H2, Graph

4. Define callback functions to update the charts based on the selected country.
- Callback + def to update titles
- Callback + def to update total_cases and total_deaths

5. Run the Dash app using the provided function for Google Colab.

In [3]:
df = pd.read_csv('/drive/My Drive/FINAL PROJECTS/Covid_19/project_1_python.csv')
df.head(2)

,index,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,hosp_patients,...,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations,population,median_age,gdp_per_capita,life_expectancy,latitude,longitude
0,0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,39835428.0,18.6,1803.987,64.83,33.0,65.0
1,1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,39835428.0,18.6,1803.987,64.83,33.0,65.0


In [4]:
# New DF for columns country, date, total_cases and total_deaths
dashboard_1 = df[['location', 'date', 'total_cases', 'total_deaths']]
dashboard_1

,location,date,total_cases,total_deaths
0,Afghanistan,2020-02-24,5.0,NaN
1,Afghanistan,2020-02-25,5.0,NaN
2,Afghanistan,2020-02-26,5.0,NaN
3,Afghanistan,2020-02-27,5.0,NaN
4,Afghanistan,2020-02-28,5.0,NaN
...,...,...,...,...
180472,Zimbabwe,2022-06-14,254155.0,5521.0
180473,Zimbabwe,2022-06-15,254387.0,5525.0
180474,Zimbabwe,2022-06-16,254502.0,5526.0
180475,Zimbabwe,2022-06-17,254753.0,5533.0


In [9]:
# Initialize the Dash app
app = dash.Dash()

# Define layout
app.layout = html.Div([
    html.H1(children='COVID-19 Dashboard', style={'text-align': 'center', }),
    html.Label('Select a location:'),
    dcc.Dropdown(
        id='location-dropdown',
        options=[{'label': location, 'value': location} for location in dashboard_1['location'].unique()],
        value=dashboard_1['location'].unique()[0]
    ),
    html.Div([
        html.Div([
            html.H2(id='cases-graph-title', style={'text-align': 'center'}),
            dcc.Graph(id='cases-graph')
        ], style={'width': '50%', 'display': 'inline-block'}),
        html.Div([
            html.H2(id='deaths-graph-title', style={'text-align': 'center'}),
            dcc.Graph(id='deaths-graph')
        ], style={'width': '50%', 'display': 'inline-block'})
    ])
])


# Define H2 titles callbacks

@app.callback(
    [Output('cases-graph-title', 'children'), Output('deaths-graph-title', 'children')],
    [Input('location-dropdown', 'value')]
)
def update_titles(selected_location):
    cases_title = f'Cumulative number of positive cases in {selected_location}'
    deaths_title = f'Cumulative number of deaths in {selected_location}'
    return cases_title, deaths_title

# Define callbacks to update total_cases graph based on selected country

@app.callback(
    Output('cases-graph', 'figure'),
    [Input('location-dropdown', 'value')]
)
def update_cases_graph(selected_location):
    filtered_df = dashboard_1[dashboard_1['location'] == selected_location]

    # Plot the line chart with colored lines
    fig = px.line(filtered_df,
                  x='date',
                  y='total_cases',
                  title='Cumulative Cases')
    return fig

# Define callbacks to update total_deaths graph based on selected country

@app.callback(
    Output('deaths-graph', 'figure'),
    [Input('location-dropdown', 'value')]
)
def update_deaths_graph(selected_location):
    filtered_df = dashboard_1[dashboard_1['location'] == selected_location]
    fig = px.line(filtered_df, x='date', y='total_deaths', title='Cumulative Deaths')
    return fig



In [10]:
# Start the Dash app in a separate thread
thread = threading.Thread(target=app.run_server)
thread.start()
output.serve_kernel_port_as_window(port=8050)

<IPython.core.display.Javascript object>